# Adaptive integrator for use in other codes
This is an adaptive integrator built specifically to integrate the very spikey functions that appear in the integrand of the Kubo-Bastin sea term (Manchon decomposition). It begins with a large broadening that is sure to produce a smooth function. The algorithm then identifies which are the points where the function varies the fastest and puts more points in the vicinity of those points. After this has been done, the broadening gets reduced and the process repeats. This makes it easier to find the spikes.<br><br>

# TO DO

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '/home/simao/projects_sync/codes/git_rgf')
from multiprocessing import Pool
import RGF_src as rgf

In [2]:
class adaptive_integrator():
    # Adaptive integrator
    
    x_integrand = -1 # positions of the points used for the integrand
    x_integral = -1 # positions of the points used for the integral
    integrand = -1  
    integral = -1 
    quiet = False
    
    state = {} # contains all the points calculated so far
    last = -1
    # def __init__(self, fun):
    #     self.fun = fun # function to be evaluated to integrate
        
        
    def calc_stuff(self, z):
        
        if not self.quiet:
            print(f"{np.real(z[0]):5.5f}",end=" ")
            
        return self.fun(z)
    
    def set_integrator(self, fun, x_integrand, x_integral):
        self.fun = fun
        self.x_integrand = x_integrand*1.0 # integrand
        self.x_integral  = x_integral*1.0 # list of fermi energies

    def add_eta(self, eta):
        # add new value of the broadening to calculate
        self.last += 1
        self.trues = [False for x in self.x_integrand]
        self.state[self.last] = [eta, self.x_integrand*1.0, self.x_integrand*0.0, self.trues]

In [5]:
def calc_one(self, tol):
    
    
    # Calculate the integral and check if it is within the tolerance
#     NE = len(self.x_integral)
#     self.integral = np.zeros(NE, dtype=complex)
#     integral_half = np.zeros(NE, dtype=complex)
#     for i,x in enumerate(self.x_integral):
#         self.integral[i] = rgf.cond(x, self.x_integrand, self.x_integral)
#         integral_half[i] = rgf.cond(x, self.x_integrand[::2], self.x_integral[::2])
        
#     if np.abs(integral_half - self.integral) < tol:
#         print("No need to calculate. Integral is within tolerance")
#         return 
    
    
    # Calculate all the points which are missing
    missing_indices = []
    missing_xs = []
    for i,(x,t) in enumerate(zip(self.x_integrand, self.trues)):
        missing_indices.append(i)
        missing_xs.append(x)
    
    missing_xs = np.array(missing_xs)
    print(missing_xs)
    print(missing_indices)

    return 
    NE1 = len(self.x_integrand)
    

    # Calculate the integrand parallelized
    zs1 = self.energies1 + eta1*1j
    bast = []
    with Pool(8) as p:
        bast = p.map(self.calc_stuff, list(np.array([z]) for z in zs1))
    self.bastingrand = np.array(bast)

    if not self.quiet:
        print("###################### ETA ",eta1,"###############################")
    else:
        print("eta", eta1)






    # Refine the integrand
    while True:

        # Identify the points with large derivatives
        der = self.bastingrand[1:] - self.bastingrand[:-1]
        ens = self.energies1[1:] - self.energies1[:-1]
        var = np.abs(der)

        # General overview
        if not self.quiet:
            fig, axs = plt.subplots(1,3,figsize=(12,4))
            axs[0].set_title("Real bastingrand")
            axs[1].set_title("Imag bastingrand")
            axs[0].plot(self.energies1, np.real(self.bastingrand))
            axs[1].plot(self.energies1, np.imag(self.bastingrand))
            axs[2].plot(self.energies1[:-1], var)
            for i in self.energies1:
                for j in range(3):
                    axs[j].axvline(i, alpha=0.001, color='red')
            plt.show()
            print("#########################################################################################################")
            print(len(self.energies1), len(self.bastingrand), np.sum(var*ens))

        # Create a new point in the middle of each problematic pair of points
        points = []
        pointsz = []
        bast = []
        count = 0
        for i in range(NE1-1):
            v = var[i]*ens[i]
            if v > tol:

                e1 = self.energies1[i]
                e2 = self.energies1[i+1]
                e = (e1+e2)/2

                count += 1

                points.append(e)
                z = e + 1j*eta1
                z = np.array([z])
                pointsz.append(z)


        # Calculate the integrand in the new set of points
        with Pool(8) as p:
            bast = p.map(self.calc_stuff, pointsz)


        # If no new points have been found within this tolerance, go to next eta
        if count == 0:    
            if not self.quiet:
                print("found no points")
            break


        # Add the new energies and integrand values to the current list
        new_energies    = np.array(points + list(self.energies1  ))
        new_bastingrand = np.array(bast   + list(self.bastingrand))
        size = len(new_energies)

        res = np.zeros([size, 2], dtype=complex)
        res[:,0] = new_energies
        res[:,1] = new_bastingrand

        if not self.quiet:
            print("size of new energies: ", size)

        new = res[res[:, 0].argsort()]

        NE1 = size
        self.energies1 = np.real(new[:,0])
        self.bastingrand = new[:,1]


        # Recalculate the integral
        self.bastin = np.zeros(NE, dtype=complex)
        for i,mu in enumerate(self.energies):
            self.bastin[i] = rgf.cond(mu, self.energies1, self.bastingrand)

        # General view of the new integral and comparison with Kubo-Greenwood
        if not self.quiet:
            fig, axs = plt.subplots(1,3,figsize=(15,4))

            axs[0].set_title("Real")
            axs[1].set_title("Imag")
            axs[2].set_title("Bastin - prev")
            axs[0].plot(self.energies, np.real(self.bastin))
            axs[0].plot(self.energies, np.real(self.prev),'--')
            axs[1].plot(self.energies, np.imag(self.bastin))
            axs[1].plot(self.energies, np.imag(self.prev),'--')
            axs[2].plot(self.energies, np.imag(self.prev-self.bastin),'--')
            plt.show()


        # If the maximum difference between two iterations is sufficiently small
        # then go to the next eta
        maxdif = np.max(np.abs(self.prev-self.bastin))
        prev = self.bastin*1.0 # save the previous value to establish a comparison in the next iteration
        if not self.quiet:
            print("MAXDIF",maxdif)

        if maxdif < 1e-2:
            print("done")
            break


    print("Reached smallest eta")
    
adaptive_integrator.calc_one = calc_one

In [6]:
def f(x, eta):
    return 1.0/(1 + eta**2)

xs = np.linspace(-1,1,20)
xs_int = np.linspace(-0.5, 0.5, 10)

grator = adaptive_integrator()
grator.set_integrator(f, xs, xs_int)
grator.add_eta(0.1)

print(grator.state)
grator.calc_one(0.1)

{0: [0.1, array([-1.        , -0.89473684, -0.78947368, -0.68421053, -0.57894737,
       -0.47368421, -0.36842105, -0.26315789, -0.15789474, -0.05263158,
        0.05263158,  0.15789474,  0.26315789,  0.36842105,  0.47368421,
        0.57894737,  0.68421053,  0.78947368,  0.89473684,  1.        ]), array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.]), [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]]}
[-1.0, -0.8947368421052632, -0.7894736842105263, -0.6842105263157895, -0.5789473684210527, -0.4736842105263158, -0.368421052631579, -0.26315789473684215, -0.1578947368421053, -0.052631578947368474, 0.05263157894736836, 0.1578947368421053, 0.26315789473684204, 0.36842105263157876, 0.4736842105263157, 0.5789473684210527, 0.6842105263157894, 0.7894736842105261, 0.894736842105263, 1.0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1